# Narupa Multiplayer Selections Test

This notebook demonstrates how to create a selection via the multiplayer key value store.

In [ ]:
from narupa.app.client import NarupaClient
from narupa.mdanalysis import frame_data_to_mdanalysis
from google.protobuf.struct_pb2 import Value, Struct
from google.protobuf.json_format import MessageToDict

## Start a Narupa IMD client

In [ ]:
client = NarupaClient()
first_frame = client.wait_until_first_frame(check_interval=0.5, timeout=10)
print(first_frame.particle_count)

In [ ]:
client.join_multiplayer("selections test")
multiplayer_client = client._multiplayer_client
player_id = multiplayer_client.player_id
print(player_id)

## Create and Modify Selection

In [ ]:
selection = client.create_selection("Selection", [0, 1, 2, 3, 4])

### Delete all selections

In [ ]:
client.clear_selections()

### Add more indices to the selection

In [ ]:
with selection.modify():
    selection.add_particles([5, 4, 3])

### Print the current selection(s)

In [ ]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

### Replace the selection indices

In [ ]:
with selection.modify():
    selection.set_particles([1,2,3])

### Clear the selection

In [ ]:
with selection.modify():
    selection.clear_particles()

### Set a selection from MDAnalysis

In [ ]:
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

### Set the rendering of a selection

In [ ]:
with selection.modify():
    selection.renderer = {
        'color': 'CornflowerBlue',
        'render': 'goodsell'
    }

# Example Use

In [ ]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.cm.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

protein = client.create_selection("Protein", [])
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))
    protein.renderer = {
        'color': {
            'type': 'residue index in entity',
            'gradient': get_matplotlib_gradient('viridis')
        },
        'render': 'goodsell'
    }
    
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))
    ligand.set_particles([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
    ligand.renderer = {
        'color': 'IndianRed',
        'particle.scale': 0.1,
        'bond.scale': 0.01,
        'render': 'liquorice'
    }
    
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True